# [NTDS'19] assignment 1: network science
[ntds'19]: https://github.com/mdeff/ntds_2019

[Eda Bayram](https://lts4.epfl.ch/bayram), [EPFL LTS4](https://lts4.epfl.ch) and
[Nikolaos Karalias](https://people.epfl.ch/nikolaos.karalias), [EPFL LTS2](https://lts2.epfl.ch).

## Students

* Team: `<your team number>`
* Students: `<your name`> (for the indivudual submission) or `<the name of all students in the team>` (for the team submission)

## Rules

Grading:
* The first deadline is for individual submissions. The second deadline is for the team submission.
* All team members will receive the same grade based on the team solution submitted on the second deadline.
* As a fallback, a team can ask for individual grading. In that case, solutions submitted on the first deadline are graded.
* Collaboration between team members is encouraged. No collaboration between teams is allowed.

Submission:
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
  Note that Networkx is imported in the second section and cannot be used in the first.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart Kernel and Run All Cells" in Jupyter.

## Objective

The purpose of this milestone is to explore a given dataset, represent it by network by constructing different graphs. In the first section, you will analyze the network properties. In the second section, you will explore various network models and find out the network model fitting the ones you construct from the dataset.

## Cora Dataset

The [Cora dataset](https://linqs.soe.ucsc.edu/node/236) consists of scientific publications classified into one of seven research fields. 

* **Citation graph:** the citation network can be constructed from the connections given in the `cora.cites` file.
* **Feature graph:** each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary and its research field, given in the `cora.content` file. The dictionary consists of 1433 unique words. A feature graph can be constructed using the Euclidean distance between the feature vector of the publications.

The [`README`](data/cora/README) provides details about the content of [`cora.cites`](data/cora/cora.cites) and [`cora.content`](data/cora/cora.content).

## Section 1: Network Properties

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

### Question 1: Construct a Citation Graph and a Feature Graph

Read the `cora.content` file into a Pandas DataFrame by setting a header for the column names. Check the `README` file.

## Feature graph adjacency matrix

In [ ]:
#Load and change name
pd_content = pd.read_csv('data/cora/cora.csv', delimiter='\t', header=None)
pd_content.rename(columns={0: 'paper_id', 1434: 'class_label'}, inplace=True)
pd_content.head()


Print out the number of papers contained in each of the reasearch fields.

**Hint:** You can use the `value_counts()` function.

In [ ]:
#Field paper count
pd_content['class_label'].value_counts()

Select all papers from a field of your choice and store their feature vectors into a NumPy array.
Check its shape.

In [ ]:
#Select the genetic algorithms and drop the class_label once it's filtered
my_field = 'Probabilistic_Methods'
features = pd_content[pd_content['class_label'] == my_field].drop('class_label', axis=1)

#Keep paper ids to select futur citation graph in next steps
paper_ids = features['paper_id']
N = features.shape[0]
features.drop('paper_id', axis=1, inplace=True)

features.shape

Let $D$ be the Euclidean distance matrix whose $(i,j)$ entry corresponds to the Euclidean distance between feature vectors $i$ and $j$.
Using the feature vectors of the papers from the field which you have selected, construct $D$ as a Numpy array. 

In [ ]:
#Calculate the euclidean distance between each pair features
feat = features.values
distance = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        distance[i, j] = np.sqrt(np.sum(np.square(feat[i,:] - feat[j, :])))

Check the mean pairwise distance $\mathbb{E}[D]$.

In [ ]:
mean_distance = distance.mean()
mean_distance

Plot an histogram of the euclidean distances.

In [ ]:
plt.figure(1, figsize=(8, 4))
plt.title("Histogram of Euclidean distances between papers")
plt.hist(distance.flatten());

Now create an adjacency matrix for the papers by thresholding the Euclidean distance matrix.
The resulting (unweighted) adjacency matrix should have entries
$$ A_{ij} = \begin{cases} 1, \; \text{if} \; d(i,j)< \mathbb{E}[D], \; i \neq j, \\ 0, \; \text{otherwise.} \end{cases} $$

First, let us choose the mean distance as the threshold.

In [ ]:
#Algorithm to create the edges
threshold = mean_distance
A_feature = distance.copy()
A_feature[A_feature < threshold] = 1
A_feature[A_feature >= threshold] = 0
A_feature -= np.identity(A_feature.shape[0])
A_feature.shape

## Citation adjacency matrix

Now read the `cora.cites` file and construct the citation graph by converting the given citation connections into an adjacency matrix.

In [ ]:
#Make the idx paper_id map
idx_map = paper_ids.reset_index(drop=True)
idx_map = pd.Series(idx_map.index, index=idx_map.values)

In [ ]:
cora_cites = np.genfromtxt('data/cora/cora.cites', delimiter='\t')

In [ ]:
cora_df = pd.DataFrame(cora_cites, columns=['cited', 'citing'])
cora_df = cora_df.astype('int')
cora_df.head()

In [ ]:
#Make the adjacency matrix for citation
A_citation = np.zeros((N, N), dtype=int)
A_citation.shape

for idx, row in cora_df.iterrows():
    i, j = int(row.cited), int(row.citing)
    if (i in idx_map.index) and (j in idx_map.index):
        A_citation[idx_map[i], idx_map[j]] = 1  # weight
        A_citation[idx_map[j], idx_map[i]] = 1  # weight to obtain an undirected network

Above are the papers that are not neither citing or cited by the other "my_field" but cite or are cited by another field paper. To overcome this issue, we suggest to padd the A_citing array by 2 times values.

Get the adjacency matrix of the citation graph for the field that you chose.
You have to appropriately reduce the adjacency matrix of the citation graph.

Check if your adjacency matrix is symmetric. Symmetrize your final adjacency matrix if it's not already symmetric.

In [ ]:
np.count_nonzero(A_citation - A_citation.transpose())

Check the shape of your adjacency matrix again.

In [ ]:
A_citation.shape

In [ ]:
np.save('data/A_citation', A_citation)
np.save('data/A_feature', A_feature)

### Question 2: Degree Distribution and Moments

What is the total number of edges in each graph?

In [ ]:
#Added to simplify
A_citation = np.load('data/A_citation.npy')
A_feature = np.load('data/A_feature.npy')

In [ ]:
#Check the shape of both graph adjacency matrices
A_citation.shape, A_feature.shape

In [ ]:
#Number of edges for each graph
num_edges_feature = int(np.sum(A_feature)/2)
num_edges_citation = int(np.sum(A_citation)/2)

print(f"Number of edges in the feature graph: {num_edges_feature}")
print(f"Number of edges in the citation graph: {num_edges_citation}")

Plot the degree distribution histogram for each of the graphs.

In [ ]:
def get_degree_distribution(A_matrix):
    return pd.DataFrame(A_matrix).sum()

In [ ]:
degrees_citation = get_degree_distribution(A_citation)
degrees_feature = get_degree_distribution(A_feature)

deg_hist_normalization = np.ones(degrees_citation.shape[0]) / degrees_citation.shape[0]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 4))
axes[0].set_title('Citation graph degree distribution')
axes[0].hist(degrees_citation, weights=deg_hist_normalization, bins=100);
axes[1].set_title('Feature graph degree distribution')
axes[1].hist(degrees_feature, weights=deg_hist_normalization, bins=100);

Calculate the first and second moments of the degree distribution of each graph.

In [ ]:
def compute_moment(degree_series, order=1):
    if order == 1:
        return np.sum(degree_series.index * degree_series.values)
    if order == 2:
        return np.sum(degree_series.index**2 * degree_series.values)
    else:
        print("this order is not implemented")

In [ ]:
cit_moment_1 = compute_moment(degrees_citation, 1)
cit_moment_2 = compute_moment(degrees_citation, 2)

feat_moment_1 = compute_moment(degrees_feature, 1)
feat_moment_2 = compute_moment(degrees_feature, 2)

print(f"1st moment of citation graph: {cit_moment_1}")
print(f"2nd moment of citation graph: {cit_moment_2}")
print(f"1st moment of feature graph: {feat_moment_1}")
print(f"2nd moment of feature graph: {feat_moment_2}")

What information do the moments provide you about the graphs?
Explain the differences in moments between graphs by comparing their degree distributions.

**Your answer here:**
The first moment is similar to the mean: on average, a node as the 1st moment value as a degree
The second moment is usually reduced by the mean in the summation process to give the variance: it gives an information on the propency of the graph to have node far from the mean.

The first graph has a low mean and low variance. Meaning that it fits quite good with the graph
The second graph graph is trange because it's a

Select the 20 largest hubs for each of the graphs and remove them. Observe the sparsity pattern of the adjacency matrices of the citation and feature graphs before and after such a reduction.

In [ ]:
#Select index of the 20 largest nodes
paper_citation_degree = pd.DataFrame(A_citation).sum()
citation_hubs = paper_citation_degree.sort_values(ascending=False).head(20).index
paper_feature_degree = pd.DataFrame(A_feature).sum()
feature_hubs = paper_feature_degree.sort_values(ascending=False).head(20).index

In [ ]:
#Set to 0 all lines and columns corresponding to hubs
A_citation_copy = pd.DataFrame(A_citation).copy()

A_citation_copy.at[citation_hubs] = 0
A_citation_copy.at[:, citation_hubs] = 0

A_feature_copy = pd.DataFrame(A_feature).copy()
A_feature_copy.at[feature_hubs] = 0
A_feature_copy.at[:, feature_hubs] = 0

In [ ]:
reduced_A_feature = A_feature_copy
reduced_A_citation = A_citation_copy

fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes[0, 0].set_title('Feature graph: adjacency matrix sparsity pattern')
axes[0, 0].spy(A_feature);
axes[0, 1].set_title('Feature graph without top 20 hubs: adjacency matrix sparsity pattern')
axes[0, 1].spy(reduced_A_feature);
axes[1, 0].set_title('Citation graph: adjacency matrix sparsity pattern')
axes[1, 0].spy(A_citation);
axes[1, 1].set_title('Citation graph without top 20 hubs: adjacency matrix sparsity pattern')
axes[1, 1].spy(reduced_A_citation);

Plot the new degree distribution histograms.

In [ ]:
reduced_degrees_feat = get_degree_distribution(reduced_A_feature)
reduced_degrees_cit = get_degree_distribution(reduced_A_citation)

deg_hist_normalization = np.ones(reduced_degrees_feat.shape[0])/reduced_degrees_feat.shape[0]


fig, axes = plt.subplots(1, 2, figsize=(16, 4))
axes[0].set_title('Citation graph degree distribution')
axes[0].hist(reduced_degrees_cit, weights=deg_hist_normalization);
axes[1].set_title('Feature graph degree distribution')
axes[1].hist(reduced_degrees_feat, weights=deg_hist_normalization);

Compute the first and second moments for the new graphs.

In [ ]:
reduced_cit_moment_1 = compute_moment(reduced_degrees_cit, 1)
reduced_cit_moment_2 = compute_moment(reduced_degrees_cit, 2)

reduced_feat_moment_1 = compute_moment(reduced_degrees_feat, 1)
reduced_feat_moment_2 = compute_moment(reduced_degrees_feat, 2)


print("Citation graph first moment:", reduced_cit_moment_1)
print("Citation graph second moment:", reduced_cit_moment_2)
print("Feature graph first moment: ", reduced_feat_moment_1)
print("Feature graph second moment: ", reduced_feat_moment_2)

Print the number of edges in the reduced graphs.

In [ ]:
# L = 1/2 * sum(ki)
L_citation = (1 / 2) * np.sum(reduced_A_citation.values)
L_feature = (1 / 2) * np.sum(reduced_A_feature.values)

print("Number of edges in citation graph: " + str(L_citation))
print("Number of edges in feature graph: " + str(L_feature))

Is the effect of removing the hubs the same for both networks? Look at the percentage changes for each moment. Which of the moments is affected the most and in which graph? Explain why.  

**Hint:** Examine the degree distributions.

In [ ]:
change_cit_moment_1 = reduced_cit_moment_1 / cit_moment_1
change_cit_moment_2 = reduced_cit_moment_2 / cit_moment_2

change_feat_moment_1 = reduced_feat_moment_1 / feat_moment_1
change_feat_moment_2 = reduced_feat_moment_2 / feat_moment_2

print("1st order momentum in citation changed by: " + str(-round((1 - change_cit_moment_1)*100, 2)) + "%")
print("2nd order momentum in citation changed by: " + str(-round((1 - change_cit_moment_2)*100, 2)) + "%")

print("1st order momentum in feature changed by: " + str(-round((1 - change_feat_moment_1)*100, 2)) + "%")
print("2nd order momentum in feature changed by: " + str(-round((1 - change_feat_moment_2)*100, 2)) + "%")

**Your answer here:**

The effect of removing the hubs is not the same for both graphs. Indeed, in the citation graph the hubs where not much connected than the mean so removing them did not change the degree distribution a lot. However, in the feature graph, this action removed a frequent degree in the high values. This action make the latter distribution closer to a poisson or power law distribution and less multivariate distribution.

In relative terms (percentage), removing the hubs does not affect the mean (1st order moment) differently in both graphs. But in the second order, the hubs affect a lot citation graph and less feature graph.

However, in absolute term, partely since the feature graph is much larger too, the first and second order moments are affected from a much larger amount of degrees than the citation graph.

All this shows us how different is the relation between hubs and non-hubs in both graph.

### Question 3: Pruning, sparsity, paths

By adjusting the threshold of the euclidean distance matrix, prune the feature graph so that its number of edges is roughly close (within a hundred edges) to the number of edges in the citation graph.

In [ ]:
threshold = mean_distance - 2.7 #After many trials, it was not possible to go closer than a difference of 117 edges
A_feature_pruned = distance.copy()
A_feature_pruned[A_feature_pruned < threshold] = 1
A_feature_pruned[A_feature_pruned >= threshold] = 0
A_feature_pruned -= np.identity(A_feature_pruned.shape[0])
num_edges_feature_pruned = np.sum(A_feature_pruned.flatten()) / 2

print(f"Number of edges in the feature graph: {num_edges_feature}")
print(f"Number of edges in the feature graph after pruning: {num_edges_feature_pruned}")
print(f"Number of edges in the citation graph: {num_edges_citation}")

Check your results by comparing the sparsity patterns and total number of edges between the graphs.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].set_title('Citation graph sparsity')
axes[0].spy(A_citation);
axes[1].set_title('Feature graph sparsity')
axes[1].spy(A_feature_pruned);

Let $C_{k}(i,j)$ denote the number of paths of length $k$ from node $i$ to node $j$. 

We define the path matrix $P$, with entries
$ P_{ij} = \displaystyle\sum_{k=0}^{N}C_{k}(i,j). $

Calculate the path matrices for both the citation and the unpruned feature graphs for $N =10$.  

**Hint:** Use [powers of the adjacency matrix](https://en.wikipedia.org/wiki/Adjacency_matrix#Matrix_powers).

In [ ]:
from numpy.linalg import matrix_power
def compute_matrix_power(matrix, n):
    '''The resulting matrix gives at position (i,j) the number of path of length n from i to j'''
    return matrix_power(matrix, n+1)

In [ ]:
def compute_path_matrix(matrix, largest_path_length):
    return sum([compute_matrix_power(matrix, i) for i in range(largest_path_length)])

In [ ]:
path_matrix_citation = compute_path_matrix(A_citation, 10)
path_matrix_feature = compute_path_matrix(A_feature, 10)

Check the sparsity pattern for both of path matrices.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 9))
axes[0].set_title('Citation Path matrix sparsity')
axes[0].spy(path_matrix_citation);
axes[1].set_title('Feature Path matrix sparsity')
axes[1].spy(path_matrix_feature, marker=1);

Now calculate the path matrix of the pruned feature graph for $N=10$. Plot the corresponding sparsity pattern. Is there any difference?

In [ ]:
path_matrix_pruned = compute_path_matrix(A_feature_pruned, 10)

plt.figure(figsize=(12, 6))
plt.title('Feature Path matrix sparsity')
plt.spy(path_matrix_pruned, marker=1);

**Your answer here:** Yes, unpruned is connected. Whereas pruned graph is not connected

Describe how you can use the above process of counting paths to determine whether a graph is connected or not. Is the original (unpruned) feature graph connected?

**Your answer here:** A graph is connected if it exists at least one path to go to any node. Visually, a graph is connected if there no blank line (except the diagonal) so yes the unpruned is connected

If the graph is connected, how can you guess its diameter using the path matrix?

**Your answer here:**  

If any of your graphs is connected, calculate the diameter using that process.

In [ ]:
diameter = 
print(f"The diameter is: {diameter}")

Check if your guess was correct using [NetworkX](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.distance_measures.diameter.html).
Note: usage of NetworkX is only allowed in this part of Section 1.

In [ ]:
import networkx as nx
feature_graph = nx.from_numpy_matrix(A_feature)
print(f"Diameter according to networkx: {nx.diameter(feature_graph)}")

## Section 2: Network Models

In this section, you will analyze the feature and citation graphs you constructed in the previous section in terms of the network model types.
For this purpose, you can use the NetworkX libary imported below.

In [ ]:
import networkx as nx

Let us create NetworkX graph objects from the adjacency matrices computed in the previous section.

In [ ]:
G_citation = nx.from_numpy_matrix(A_citation)
print('Number of nodes: {}, Number of edges: {}'. format(G_citation.number_of_nodes(), G_citation.number_of_edges()))
print('Number of self-loops: {}, Number of connected components: {}'. format(G_citation.number_of_selfloops(), nx.number_connected_components(G_citation)))

In the rest of this assignment, we will consider the pruned feature graph as the feature network.

In [ ]:
G_feature = nx.from_numpy_matrix(A_feature_pruned)
print('Number of nodes: {}, Number of edges: {}'. format(G_feature.number_of_nodes(), G_feature.number_of_edges()))
print('Number of self-loops: {}, Number of connected components: {}'. format(G_feature.number_of_selfloops(), nx.number_connected_components(G_feature)))

### Question 4: Simulation with Erdős–Rényi and Barabási–Albert models

Create an Erdős–Rényi and a Barabási–Albert graph using NetworkX to simulate the citation graph and the feature graph you have. When choosing parameters for the networks, take into account the number of vertices and edges of the original networks.

The number of nodes should exactly match the number of nodes in the original citation and feature graphs.

In [ ]:
assert len(G_citation.nodes()) == len(G_feature.nodes())
n = len(G_citation.nodes())
n

The number of match shall fit the average of the number of edges in the citation and the feature graph.

In [ ]:
m = np.round((G_citation.size() + G_feature.size()) / 2)
m

How do you determine the probability parameter for the Erdős–Rényi graph?

**Your answer here:** The probability that two nodes are connected is the number of edges divided by the number of nodes

In [ ]:
#The probability that a node is connected is the number of edges divided by the number of nodes
p = m / n
G_er = nx.erdos_renyi_graph(n, p)

Check the number of edges in the Erdős–Rényi graph.

In [ ]:
print('My Erdos-Rényi network that simulates the citation graph has {} edges.'.format(G_er.size()))

How do you determine the preferential attachment parameter for Barabási–Albert graphs?

**Your answer here:** a random selected node connects to node i with proba Pi(k_i) = k_i/sum(k_j)

In [ ]:
#Assuming at each timestep the new node connects to the same number of nodes, we have to devide the total number of edges
#by the number of nodes
q = int(p)
G_ba = nx.barabasi_albert_graph(n, q)

Check the number of edges in the Barabási–Albert graph.

In [ ]:
print('My Barabási-Albert network that simulates the citation graph has {} edges.'.format(G_ba.size()))

### Question 5: Giant Component

Check the size of the largest connected component in the citation and feature graphs.

In [ ]:
giant_citation = max(nx.connected_component_subgraphs(G_citation), key=len)
print('The giant component of the citation graph has {} nodes and {} edges.'.format(giant_citation.number_of_nodes(), giant_citation.size()))

In [ ]:
giant_feature = max(nx.connected_component_subgraphs(G_feature), key=len)
print('The giant component of the feature graph has {} nodes and {} edges.'.format(giant_feature.number_of_nodes(), giant_feature.size()))

Check the size of the giant components in the generated Erdős–Rényi graph.

In [ ]:
giant_er = max(nx.connected_component_subgraphs(G_er), key=len)
print('The giant component of the Erdos-Rényi network has {} nodes and {} edges.'.format(giant_er.number_of_nodes(), giant_er.size()))

Let us match the number of nodes in the giant component of the feature graph by simulating a new Erdős–Rényi network.
How do you choose the probability parameter this time? 

**Hint:** Recall the expected giant component size from the lectures.

**Your answer here:** The parameter is the probabibility for 2 nodes in the hub to be attached by one edge => # edges in the hub / # nodes in the hub

In [ ]:
p_new = giant_feature.number_of_edges() / giant_feature.number_of_nodes()
G_er_new = nx.erdos_renyi_graph(n, p_new)

Check the size of the new Erdős–Rényi network and its giant component.

In [ ]:
print('My new Erdos Renyi network that simulates the citation graph has {} edges.'.format(G_er_new.size()))
giant_er_new = max(nx.connected_component_subgraphs(G_er_new), key=len)
print('The giant component of the new Erdos-Rényi network has {} nodes and {} edges.'.format(giant_er_new.number_of_nodes(), giant_er_new.size()))

### Question 6: Degree Distributions

Recall the degree distribution of the citation and the feature graph.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes[0].set_title('Citation graph')
citation_degrees = degrees_citation
axes[0].hist(citation_degrees);
axes[1].set_title('Feature graph')
feature_degrees = degrees_feature
axes[1].hist(feature_degrees);

What does the degree distribution tell us about a network? Can you make a prediction on the network model type of the citation and the feature graph by looking at their degree distributions?

**Your answer here:** Most part of the nodes are low degree. We could guess a poisson distribution or a power law distribution.

Now, plot the degree distribution historgrams for the simulated networks.

In [ ]:
import collections
def degree_sequence(G):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
    # print "Degree sequence", degree_sequence
    degreeCount = collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())
    return deg, cnt

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
axes[0].set_title('Erdos-Rényi network')
er_degrees, _ = degree_sequence(G_er)
axes[0].hist(er_degrees);
axes[1].set_title('Barabási-Albert network')
ba_degrees, _ = degree_sequence(G_ba)
axes[1].hist(ba_degrees);
axes[2].set_title('new Erdos-Rényi network')
er_new_degrees, _ = degree_sequence(G_er_new)
axes[2].hist(er_new_degrees);

In terms of the degree distribution, is there a good match between the citation and feature graphs and the simulated networks?
For the citation graph, choose one of the simulated networks above that match its degree distribution best. Indicate your preference below.

**Your answer here:** 

You can also simulate a network using the configuration model to match its degree disctribution exactly. Refer to [Configuration model](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html#networkx.generators.degree_seq.configuration_model).

Let us create another network to match the degree distribution of the feature graph. 

In [ ]:
G = nx.configuration_model(degrees_citation)
plt.hist(degree_sequence(G))

In [ ]:
G_config = nx.configuration_model(feature_degrees) 
print('Configuration model has {} nodes and {} edges.'.format(G_config.number_of_nodes(), G_config.size()))

Does it mean that we create the same graph with the feature graph by the configuration model? If not, how do you understand that they are not the same?

**Your answer here:** 

### Question 7: Clustering Coefficient

Let us check the average clustering coefficient of the original citation and feature graphs. 

In [ ]:
nx.average_clustering(G_citation)

In [ ]:
nx.average_clustering(G_feature)

What does the clustering coefficient tell us about a network? Comment on the values you obtain for the citation and feature graph.

**Your answer here:**

Now, let us check the average clustering coefficient for the simulated networks.

In [ ]:
nx.average_clustering(G_er)

In [ ]:
nx.average_clustering(G_ba)

In [ ]:
nx.average_clustering(nx.Graph(G_config))

Comment on the values you obtain for the simulated networks. Is there any good match to the citation or feature graph in terms of clustering coefficient?

**Your answer here:**

Check the other [network model generators](https://networkx.github.io/documentation/networkx-1.10/reference/generators.html) provided by NetworkX. Which one do you predict to have a better match to the citation graph or the feature graph in terms of degree distribution and clustering coefficient at the same time? Justify your answer.

**Your answer here:**

If you find a better fit, create a graph object below for that network model. Print the number of edges and the average clustering coefficient. Plot the histogram of the degree distribution.

In [ ]:
# Your code here.

Comment on the similarities of your match.

**Your answer here:**